In [48]:
# 27.1. Building trees

class Tree:
    def __init__(self, cargo, left=None, right=None):
        self.cargo = cargo
        self.left = left
        self.right = right
        
    def __str__(self):
        return str(self.cargo)

In [49]:
# One way to build a tree is from the bottom up. Allocate the child nodes first:

left = Tree(2)
right = Tree(3)

In [50]:
# Then create the parent node and link it to the children:
tree = Tree(1, left, right)

In [51]:
# We can write this code more concisely by nesting constructor invocations:
tree = Tree(1, Tree(2), Tree(3))

In [52]:
def total(tree):
    if tree is None: return 0
    return total(tree.left) + total(tree.right) + tree.cargo

In [53]:
total(tree)

6

In [54]:
# 27.3. Expression trees
#  1 + 2 * 3

tree = Tree("+", Tree(1), Tree("*", Tree(2), Tree(3)))

In [55]:
# 27.4. Tree traversal
def print_tree(tree):
    """traversing a tree is called a preorder"""
    if tree is None: return
    print(tree.cargo, end=" ")
    print_tree(tree.left)
    print_tree(tree.right)

In [56]:
print_tree(tree)

+ 1 * 2 3 

In [57]:
def print_tree_postorder(tree):
    """This order of traversal is called postorder"""
    if tree is None: return
    print_tree_postorder(tree.left)
    print_tree_postorder(tree.right)
    print(tree.cargo, end=" ")

In [58]:
print_tree_postorder(tree)

1 2 3 * + 

In [59]:
def print_tree_inorder(tree):
    """inorder"""
    if tree is None: return
    print_tree_inorder(tree.left)
    print(tree.cargo, end=" ")
    print_tree_inorder(tree.right)

In [60]:
print_tree_inorder(tree)

1 + 2 * 3 

In [61]:
# If we do an inorder traversal and keep track of what level in the tree we are on, 
# we can generate a graphical representation of a tree:

def print_tree_indented(tree, level=0):
    if tree is None: return
    print_tree_indented(tree.right, level+1)
    print(" " * level + str(tree.cargo))
    print_tree_indented(tree.left, level+1)

In [62]:
print_tree_indented(tree)

  3
 *
  2
+
 1


In [63]:
def get_token(token_list, expected):
    if token_list[0] == expected:
        del token_list[0]
        return True
    return False

In [67]:
def get_number(token_list):
    x = token_list[0]
    if type(x) != type(0): return None
    del token_list[0]
    return Tree(x, None, None)

In [68]:
token_list = [9, 11, "end"]
x = get_number(token_list)
print_tree_postorder(x)

9 

In [69]:
print(token_list)

[11, 'end']


In [70]:
def get_product(token_list):
    a = get_number(token_list)
    if get_token(token_list, "*"):
#         b = get_number(token_list) # This line changed
        b = get_product(token_list)
        return Tree("*", a, b)
    return a

In [71]:
token_list = [9, "*", 11, "end"]
tree = get_product(token_list)
print_tree_postorder(tree)

9 11 * 

In [72]:
token_list = [9, "+", 11, "end"]
tree = get_product(token_list)
print_tree_postorder(tree)

9 

In [73]:
token_list = [2, "*", 3, "*", 5 , "*", 7, "end"]
tree = get_product(token_list)
print_tree_postorder(tree)

2 3 5 7 * * * 

In [74]:
def get_sum(token_list):
    a = get_product(token_list)
    if get_token(token_list, "+"):
        b = get_sum(token_list)
        return Tree("+", a, b)
    return a

In [75]:
token_list = [9, "*", 11, "+", 5, "*", 7, "end"]
tree = get_sum(token_list)
print_tree_postorder(tree)

9 11 * 5 7 * + 

In [83]:
# Modify get_number to handle subexpressions

def get_number(token_list):
    if get_token(token_list, "("):
        x = get_sum(token_list)
        if not get_token(token_list, ")"):
            raise ValueError('Missing close parenthesis')
        return x
    else:
        x = token_list[0]
        if type(x) != type(0): return None
        del token_list[0]
        return Tree(x, None, None)

In [84]:
token_list = [9, "*", "(", 11, "+", 5, ")", "*", 7, "end"]
tree = get_sum(token_list)
print_tree_postorder(tree)

9 11 5 + 7 * * 

In [85]:
# 27.7. The animal tree

def yes(ques):
    ans = input(ques).lower()
    return ans[0] == "y"

def animal():
    # Start with a singleton
    root = Tree("bird")
    
    # Loop until user quits
    while True:
        print()
        if not yes("Are you thinking of an animal? "): break
            
        # Walk the tree
        tree = root
        while tree.left is not None:
            prompt = tree.cargo + "? "
            if yes(prompt):
                tree = tree.right
            else:
                tree = tree.left
        
        # make a guess
        guess = tree.cargo
        prompt = "Is it a " + guess + "? "
        if yes(prompt):
            print("I rule!")
            continue
            
        # Get new information
        prompt = "What is the animal's name? "
        animal = input(prompt)
        prompt = "What question would distinguish a {0} from a {1}? "
        question = input(prompt.format(animal, guess))
        
        # Add new information to the tree
        tree.cargo = question
        prompt = "If the animal were {0} the answer would be? "
        if yes(prompt.format(animal)):
            tree.left = Tree(guess)
            tree.right = Tree(animal)
        else:
            tree.left = Tree(animal)
            tree.right = Tree(guess)

In [87]:
animal()


Are you thinking of an animal? y
Is it a bird? y
I rule!

Are you thinking of an animal? y
Is it a bird? n
What is the animal's name? cat
What question would distinguish a cat from a bird? bird can fly
If the animal were cat the answer would be? n

Are you thinking of an animal? n
